# Artificial Intelligence - Fall 2022 - Laboratory 06

## _Searching algorithms for optimal decision-making in game theory and AI_


## Introduction

In gaming tehory, the _decision-making process_ relies on the searching algorithm guiding the investigation of the search-space.

Today's challenge sets the **MinMax Algorithm** as the main character of a  two-player game.

Using tic-tac-toe as an example, the algorithm should compute the next best move by evaluating the utility of the board.

## From definitions to know-how

In [2]:
# Useful libraries:
from collections import defaultdict
import random
import math
import functools 

The mathematical representation of the problem gets an intuitive design `class Board` holding the legal states and possible moves for the allowed positions.

Pythonic speaking, `class Board` overrides the built-in subclass `defaultdict` from `class collections`.

To manage the empty fields or to assign the corresponding values for each position, the method `__missing__` from `defaultdict`
represents a better alternative than using a traditional dictionary.

The structure of the board is:
`{(coordinates_as_tuple) : attributes}`

Where the attributes element might be:
* the player which assigns X or O on the board;
* the dimensions of the board, given by width and height;
* keywords arguments to store the **utility value** meaning the evaluation function for this problem.

### Task 0

Build the `class Board`.

In [3]:
class Board(defaultdict):
    empty = '.'
    used = '#'
    
    def __init__(self, width=3, height=3, current_player=None, **kwds):
        self.__dict__.update(width=width, height=height, current_player=current_player, **kwds)
 
    def __missing__(self, pos):
        """
        Given the position of a cell, verify if its coordinates are within the
        boundaries of the board and mark the cell as an empty square.
        Otherwise, the cell will be marked as `used`.
        
        """
        if(pos[0]<=self.width)and(pos[1]<=self.height):
            return Board.empty
        else:
            return Board.used        


            
    def __hash__(self): 
        """
        Hash method stores the instances in hash tables.
        """
        return hash(tuple(sorted(self.items()))) + hash(self.current_player)
    
    def __repr__(self):
        def row(y): 
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'
    
    def update_board(self, changes: dict, **kwds) -> 'Board':
        """
        Update the board with the new changes of each cell.
        """
        self.update(changes)
        return self

dicta=Board(3,3,{(0,0):2})
dictador={(0,0):'X',(0,1):'O',(0,2):'X'}
dicat2=Board(3,3)
pos=(0,0)

player='X'
pos=(0,0)
new_dict=dicat2.update_board({pos:player})
dicat2[(0,2)]





'.'

For this problem the utility value can be:

* _-1_ if player that seeks minimum wins;
* _0_ if it's a tie;
* _1_ if player that seeks maximum wins.

The abstract `class TicTacToe` receives a 3X3 board, where a game ends if there:

* is a vertical win or,
* is a main diagonal win or,
* is a second diagonal win or,
* is a horizontal win,
* are no empty squares. 

The parameter k is responsible for counting the number of similar symbols (_X_ or _O_) are in a row.

### Task 1

Find the number of _X_ or _O_ placed in a row.

In [4]:
def k_in_row(board, player, square, k):
    """
    Helper function to count if the player has similar symbols in a line.
    The player variable represents the player we are counting for (X or O in this case).
    The square variable can be used to only check lines that contain a specific position.
    You can choose to remove the "square" parameter from the function definition.
    """
    x=square[0]
    y=square[1]
    ## Define the target zone by indexes
    index=(k-1)
    max_index=(k+1)
    # Extract the rows
    # Vertical
    up_row=[player for j in  range(y-index,y+1) if board[(x,j)]==player]
    # Bottom
    bottom_row=[player for j in range(y,y+index+1) if board[(x,j)]==player]
    #Vertical
    vertical_row=[player for j in  range(y-index,y+index+1) if board[(x,j)]==player]
    # Left
    Left_row=[player for i in range(x-index,x+1) if board[(i,y)]==player]
    #Right
    Right_row=[player for i in range(x,x+1+index) if board[(i,y)]==player]
    #Horizontal
    Horizontal_row=[player for i in range(x-index,x+index+1) if board[(i,y)]==player]
    #Right_upper_diagonal
    Right_upper_diag=[player for _ in range(k) if board[(x+_,y-_)]==player]
    #Right_bottom_diagonal
    Right_bottom_diag=[player for _ in range(k) if board[(x+_,y+_)]==player]
    #Right Diagonal
    Right_diag=[player for _ in range(-index+1,index) if board[(x+_,y-_)]==player]
    #Left_bottom diag
    Left_bottom_diag=[player for _ in range(k) if board[(x-_,y+_)]==player]
    #Left upper diag
    Left_upper_diag=[player for _ in range(k) if board[(x-_,y-_)]==player]
    #Left diag
    Left_diag=[player for _ in range(-index+1,index) if board[(x+_,y+_)]==player]

    total_row=[len(up_row),len(vertical_row),len(Right_upper_diag),len(Right_diag),len(Right_row),len(Horizontal_row),
     len(Right_bottom_diag),len(bottom_row),len(Left_bottom_diag),len(Left_row),len(Left_diag),len(Left_upper_diag)]
    return total_row






    

    

In [5]:
board=Board(3,3)
print(board)
board.update_board({(1,0):'X'})
board.update_board({(0,2):'X'})
print(board)
row=k_in_row(board,'X',(1,1),3)
print(row)



. . .
. . .
. . .

. X .
. . .
X . .

[1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]


### Task 2

Build the `class TicTacToe`:

In [91]:
class TicTacToe:
    
    def __init__(self, height=3, width=3, k=3):
        self.k =k
        self.height=height
        self.width=width
        self.squares = {(x,y) for x in range(self.width) for y in range(self.height) }
                      
        self.initial =Board(self.width,self.height,'X')
 
    def actions(self, board):
        """
        Define the possible moves for the allowed positions.
        Hint: Remember that you have all the positions on the board in self.squares and
        all the occupied positions can be obtained from the board parameter. Set operations
        are easily handled by python.
        """
        moves={}
        for square in self.squares:
            player=board[square]
            if player =='.':
                moves[square]=board.current_player
        return moves

    def utility(self, board, player):
        
        _player='X'
        rival='O'
        player_utility=0

        for pos in self.squares:
            #print('pos,',pos)
            player_rows=k_in_row(board,_player,pos,self.k)
            #print('player row',player_rows)
            if 3 in player_rows:
                    player_utility=-1
            rival_rows=k_in_row(board,rival,pos,self.k)
            #print('rival row',rival_rows)
            if 3 in rival_rows:
                player_utility=1

        self.util = player_utility
        return player_utility

    def make_move(self, board, square):
        """
        Update the board in case the current player (board.current_player) places their symbol in the given square. 
        Afterwards, update the board's utility function with the corresponding symbol for each player.
        """
        player = board.current_player
        board.update_board({square:player})
        board.utility=self.utility(board,board.current_player)
        if board.current_player=='X':
            board.current_player='O'
        else:
            board.current_player='X'
        return board
    
    def end(self, board):
        """
        Checks if the game came to an end (we have an utility value for the board or there is a draw).
        """
        utility=self.utility(board,board.current_player)
        if (utility)!=0:
            return(True,utility)
        else:
            if self.actions(board)=={}:
                return ( True,0)
            else:
                return (False,0)
        

 
    def draw_board(self, board):
        print(board)

In [6]:
tic=TicTacToe(3,3,3)
board=tic.initial
tic.make_move(board,(0,1))
board.current_player
tic.actions(board)
tic.make_move(board,(2,2))
tic.make_move(board,(2,0))
tic.utility(board,'X')
tic.make_move(board,(1,2))
tic.make_move(board,(0,0))
tic.make_move(board,(0,2))
print('aqui')
print(tic.utility(board,'O'))




aqui
-1


In [60]:
d1={'X':1}
d2={'X':1}
if d1['X']==d2['0']:
    print(0)

KeyError: '0'

In [7]:
def random_player(game, state):
    """
    We define a player that always uses random moves.
    This will be the challenger for our algorithm.
    """
    return random.choice(list(game.actions(state)))

game=TicTacToe(3,3,3)
board=game.initial

random_player(game,board)

(0, 2)

In [8]:
def player(search_algorithm):
    """
    We define a general player that uses a strategy (search algorithm).
    Given a search algorithm, the player uses the (game, state) values to return an optimal move.
    The search_algorithm function will give us: utility_value, action_to_take.
    We only return the chosen action.
    """
    return lambda game, state: search_algorithm(game, state)[1]

### Task 3

To define the actions of the problem, we use the `play_game` function receiving the current game to play and a strategy.
The strategy itself reduces to a dictionary with the following structure:
```
{player_as_key : strategy_function}
```
where `strategy_function` can be called by: `strategy_function(state, game)`

Example:
```
X: random_player -> random_player(state, game)
O: player(minmax_search) -> minmax_search(game,state)[1] (returned by the above lambda expression)
```

The `strategy_function` returns the action for each round and the current state of each player.

In [99]:
def play_game(game, strategies: dict):
    board=game.initial
    print('Llama X')
    print(board)
    flag=False
    while flag==False:
        print(board)
        move=strategies[board.current_player](game,board)
        print('{}=>{}'.format(board.current_player,move))
        board=game.make_move(board,move)
        print(board)
        flag=game.end(board)[0] 
        
        
    utility=game.end(board)[1]  
    return utility

play_game(TicTacToe(),dict(X=random_player,O=random_player))
    


Llama X
. . .
. . .
. . .

. . .
. . .
. . .

X=>(0, 2)
. . .
. . .
X . .

. . .
. . .
X . .

O=>(1, 1)
. . .
. O .
X . .

. . .
. O .
X . .

X=>(1, 0)
. X .
. O .
X . .

. X .
. O .
X . .

O=>(2, 2)
. X .
. O .
X . O

. X .
. O .
X . O

X=>(0, 1)
. X .
X O .
X . O

. X .
X O .
X . O

O=>(0, 0)
O X .
X O .
X . O



1

In [73]:
tic=TicTacToe()
board=tic.initial
tic.make_move(board,(0,0))
tic.make_move(board,(0,1))
tic.make_move(board,(0,2))
tic.make_move(board,(1,1))
tic.make_move(board,(1,2))
tic.make_move(board,(2,1))
tic.make_move(board,(0,1))

print(board.current_player)
print(board)
tic.utility(board,'O')



O
X . .
X O O
X X .



1

## Min-Max Algorithm

### Task 4

Build the search game tree to determine the best move using:

* the `max_value(state)` function in which the AI's strategy is to _maximise_ its score while the opponent's score minimises;
* the `min_value(state)` function in which the human's strategy is to _minimise_ AI's score.

In [55]:
# Set a value worse than the worst case:
infinity = math.inf
import copy

def minimax_search(game, state):
    player = state.current_player

    def max_value(state):
        if game.end(state)[0]==True:
            return (-game.utility(state,state.current_player),(0,0)) # the positipn doesnt matter
        else:
            moves=game.actions(state)
            puntuation=[]
            for pos in moves:
                new_board=copy.deepcopy(state)
                new_board.current_player=state.current_player
            
                game.make_move(new_board,pos)
                mini=min_value(new_board)
                puntuation.append((mini[0],pos))
            
        maximum=-infinity
        max_position=(0,0)
        for values,position in puntuation:
            if values> maximum:
                maximum=values
                max_position=position
        
        return (maximum,max_position)
        
       # return value, move

    def min_value(state):
        if game.end(state)[0]==True:
            return (game.utility(state,state.current_player),(0,0)) # the positipn doesnt matter
        else:
            moves=game.actions(state)
            puntuation=[]
            for pos in moves:
                new_board=copy.deepcopy(state)
                new_board.current_player=state.current_player
                game.make_move(new_board,pos)
                max=max_value(new_board)
                puntuation.append((max[0],pos))
                
            
        minimum=infinity
        min_position=(0,0)
        for values,position in puntuation:
            if values< minimum:
                minimum=values
                min_position=position
        
        return (minimum,min_position)
    return max_value(state)

The output of the game should look like:

In [92]:
play_game(TicTacToe(), dict(X=random_player, O=player(minimax_search))).utility

NameError: name 'minimax_search' is not defined